## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test)

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [5]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), activation='relu', input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3,3), activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128)) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10, activation='softmax'))
classifier.summary()

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=128,epochs=10, validation_data=(x_test, y_test))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
batch_normalization_3 (Batch (None, 13, 13, 64)        256       
_________________________________________________________________
flatten_1 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[3.1964732e-03, 9.6735971e-07, 6.8506650e-03, 3.1750575e-01,
        6.2130734e-02, 9.0394862e-02, 2.0129808e-06, 1.1023843e-03,
        5.1881611e-01, 5.2138529e-09]], dtype=float32)

In [7]:
input_example=(x_test-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[3.3796888e-03, 1.6590083e-06, 5.9577767e-03, ..., 1.0560330e-03,
        5.3660274e-01, 1.0279092e-08],
       [3.1147848e-03, 1.0726378e-06, 6.5056924e-03, ..., 9.8511134e-04,
        5.2155745e-01, 6.2221353e-09],
       [3.1438705e-03, 1.0539562e-06, 6.5074214e-03, ..., 9.7545987e-04,
        5.4881138e-01, 6.6219479e-09],
       ...,
       [3.2361750e-03, 1.2936861e-06, 5.8896891e-03, ..., 1.2528724e-03,
        5.1784158e-01, 7.5246964e-09],
       [2.8532126e-03, 1.3028185e-06, 7.3596761e-03, ..., 1.2025357e-03,
        4.5587754e-01, 7.2918369e-09],
       [3.4317826e-03, 1.1698146e-06, 6.6132369e-03, ..., 1.0963291e-03,
        5.0615925e-01, 7.7988602e-09]], dtype=float32)